# Open AI Function Tools

## Introduction

1. Different client models with openai SDK

2. Structured Outputs

3. Guardrails

In [37]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, Field

from typing import Dict
from IPython.display import display, Markdown

In [8]:
load_dotenv(override=True)

True

In [9]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key exists and begins gsk_


### Using Other Models in OpenAI SDK

* Ensure to use OpenAI compatible endpoints

In [10]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [11]:

# deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

# deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

#### Set 3 Instructions and Assign them 3 Different Agents Respectively

In [12]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [13]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=gemini_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

#### Convert the Agents to Tools

* Converting an agent to a tool (using `.as_tool` method) creates a tool that wraps the agent that's is being converted 
* Since tools are used to call an agent, every time this tool is called, it will call the agent it wraps
---
* Writting email tools

In [14]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [15]:
# combine all the email writer together
tools = [tool1, tool2, tool3]

* Tool that sends email

In [16]:
# def send_test_email():
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("olaolagab74@gmail.com")  
#     to_email = To("gabrielagbolahan@yahoo.com")
#     content = Content("text/plain", "This is an important test email")
#     mail = Mail(from_email, to_email, "Test email", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     print(response.status_code)

In [17]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("olaolagab74@gmail.com")  
    to_email = To("gabrielagbolahan@yahoo.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
    return {"status": "success"}

* Subject tool: Select the best email of the 3 emails returned by the 3 previous agents
* Convert the text email body of subject tool to HTML

In [18]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=gemini_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=gemini_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [19]:
email_tools = [subject_tool, html_tool, send_html_email]

Manager in charge of all the agents
* formats email
* and sends email

But this email agent will be handed over this task to do by the sales manager since the sales manager is the one using this tool
not send the email directly. I will hand over to another agent that will actually convert the email and also send it.

In [20]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=gemini_model,
    handoff_description="Convert an email to HTML and send it")

In [21]:
handoffs = [emailer_agent]

* The sales manager that use the email tools to generate cold sales email and picks the best
* After picking the best, it hands over the selected email to email agent to convert the email to HTML and then send it.

In [22]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model)


In [23]:

message = "Send out a cold sales email addressed to Dear CEO from Gabriel. \
Clsing remarks should be your sincerely followed my company's name and current date"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

202


#### Check out the trace:

This is only application for Open AI models only:
* https://platform.openai.com/traces

### Guardrails

Guardrails in AI refers to techniques and tools used to ensure that AI systems behave safely, reliably, and ethically. It is basically and agent for that purpose.

* Here, gaurd email generated by LLM to ensure it does not contain sender name (a string)
* Guardrails can only validate the first input before it is fed into the system, and final output output of an agentics system.

In [24]:
# guardrails schema
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    "Detect name in the email and flag it"
    name: str
    "Name detedted in the email"

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [25]:
# this will pass the message to safeguard into huardrail and run the guardrail agent

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

#### Check out the trace:

https://platform.openai.com/traces

In [20]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

# Open AI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

* `WebSearchTool` lets an agent search the web.  
* `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
* `ComputerTool`: allows automating computer use tasks like taking screenshots and clicking.

### Research Assistant
* Using OpenAI `WebSearchTool`
* Cost of `WebSearchTool` is found here: https://platform.openai.com/docs/pricing

In [56]:
from agents import WebSearchTool, gen_trace_id
from agents.model_settings import ModelSettings
import asyncio

#### Agents

* Search Agent: uses `WebSearchTool` tool

In [31]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name = "search assistant",
    instructions = INSTRUCTIONS,
    tools = [WebSearchTool(search_context_size='low')],
    model = "gpt-4o-mini",
    model_settings = ModelSettings(tool_choice="required")
)

Executing the agent and tracking it with the trace contect manager:

In [34]:
search_message = "Top 5 AI agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, search_message)

display(Markdown(result.final_output))

As of July 2025, several AI agent frameworks have emerged as leaders in the field, each offering unique features tailored to various development needs.

**1. LangChain**: A Python-based framework renowned for its modular design, LangChain enables developers to build complex, multi-step workflows by integrating prompts, models, memory, and external tools. It supports a wide range of large language models (LLMs) and offers extensive integrations, making it ideal for applications requiring intricate logic and diverse functionalities. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**2. AutoGen**: Developed by Microsoft, AutoGen is an open-source framework designed to facilitate the creation of autonomous, event-driven systems, particularly multi-agent systems. It allows for seamless communication and collaboration among multiple AI agents, enabling the development of sophisticated conversational AI agents capable of solving complex problems through coordinated efforts. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))

**3. CrewAI**: CrewAI is a lightweight Python framework focused on building collaborative agent systems. It emphasizes the creation of teams of agents with defined roles and responsibilities, simplifying the orchestration of multi-agent systems and enhancing collaborative task execution. This framework is particularly useful for applications requiring coordinated efforts among specialized agents to achieve shared goals. ([radarmagazine.com](https://www.radarmagazine.com/top-5-ai-agent-frameworks-it-executives-should-be-watching-in-2025/?utm_source=openai))

**4. LlamaIndex**: LlamaIndex is an open-source data orchestration framework designed for developing generative AI and AI agent solutions. It features ready-to-use agents and tools, along with workflows that enable the development of multi-agent systems. LlamaIndex streamlines context augmentation for generative AI use cases by utilizing a Retrieval-Augmented Generation (RAG) pipeline, integrating multiple tools and functionalities to simplify the development process. ([antiersolutions.com](https://www.antiersolutions.com/blogs/top-ai-agent-frameworks-to-watch-in-2025-a-complete-guide/?utm_source=openai))

**5. AgentLite**: AgentLite is a lightweight, user-friendly library for building and advancing task-oriented LLM agent systems. It simplifies the process of creating and evaluating new reasoning strategies and agent architectures, facilitating research and development in the field of AI agents. ([arxiv.org](https://arxiv.org/abs/2402.15538?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the advancement of autonomous and intelligent systems across various applications. 

---
**Structuring the respsonse of Search Asistant Using Pydantic schema and description**

* Agent that plans the research:

In [48]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

In [1]:

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

NameError: name 'BaseModel' is not defined

In [50]:

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:
# for each item in the list
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


* Writer Agent

In [51]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

* Email Agent: will use the send email function tool

In [52]:

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="EmailAgent",
    instructions=INSTRUCTIONS,
    tools=[send_html_email],
    model="gpt-4o-mini",
)


#### Functions to Excute Agents

* The following unctions will plan and execute the search, using `planner_agent` and `search_agent`

In [53]:

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan and run concurrently """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results


async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

* Functions to write report for the searched items using the `writer_agent` and also send email using the `email_agent`

In [54]:

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output


async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report
    

#### Execute Research

In [57]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
202
Email sent
Hooray!
